# Trigger ArcticDEM Download 

* run with arcticdem environment: aicore has geopandas/pyproj issues

In [ ]:
import ee
import os
import geopandas as gpd
import glob
%config Completer.use_jedi = False
from modules.utils import *
import datetime
import shutil
import tqdm

In [ ]:
ee.Initialize()

### Settings

#### Paths

In [ ]:
FILEPATH = r'vector/RapidEye_Tiles.shp'
TS_FIELD = 'dem_order'

#### Setup GEE Datasets 

In [ ]:
slope = get_ArcticDEM_slope()

In [ ]:
rel_el_100 = get_ArcticDEM_rel_el(kernel_size=100)

#### Setup data
* faster loading?
* https://geopandas.org/en/stable/docs/user_guide/io.html
```python
gdf = geopandas.read_file(
    geopandas.datasets.get_path("naturalearth_lowres"),
    ignore_fields=["iso_a3", "gdp_md_est"],
)
```

In [ ]:
gdf = gpd.read_file(FILEPATH)#.to_crs('EPSG:4326')

### Manual Selection

**open QGIS, select tiles and run the follwing code in the python console:**

layer = qgis.utils.iface.activeLayer()

selected_features = layer.selectedFeatures()

[f['tile_id'] for f in selected_features]

In [ ]:
field = 'tile_id'
#list tile_ids here
idxs = []

### Auto selection of active and incomplete Locations 

#### Query where
ArtcicDEM is not processed
Planet is downloaded

Temporary

In [ ]:
existing_planet_data = gdf[['pl_2018','pl_2019','pl_2020','pl_2021', 'pl_2022']].any(axis=1)
field = 'tile_id'
idxs = list(gdf[existing_planet_data & gdf.dem_order.isna()][field].values)

In [ ]:
len(idxs)

### Start Processing DEM Tiles

In [ ]:
tasks_slope = []
tasks_el = []

for idx in idxs:
    # need to drop column: 'downloaded which may contain NaN values'
    gdf_filt = gdf.query(f'{field} == {idx}')#.drop(columns='downloaded')
    tasks_slope.append(download_ee2(gdf_filt, slope, GDrive_dir='AI-CORE/slope', scale=2,  suffix='_slope', field=field))
    tasks_el.append(download_ee2(gdf_filt, rel_el_100, GDrive_dir='AI-CORE/elevation', scale=2,  suffix='_relative_elevation_100', field=field))
    
    gdf.loc[gdf_filt.index, TS_FIELD] = datetime.datetime.now().isoformat()

#### Show Running tasks 

In [ ]:
print('Running Tasks:')
for tasks in [tasks_el, tasks_slope]:
    print([t.status()['description'] for t in tasks if t.status()['state'] == 'RUNNING'])

#### Show finished tasks 

In [ ]:
print('Completed Tasks:')
for tasks in [tasks_el, tasks_slope]:
    print([t.status()['description'] for t in tasks if t.status()['state'] == 'COMPLETED'])
#[t.status()['description'] for t in tasks if t.status()['state'] == 'COMPLETED']

#### Save updated gdf 

In [ ]:
gdf.to_file(FILEPATH)

#### control

In [ ]:
gdf.dropna(subset=['dem_order'], axis=0)

### Move processed files

In [ ]:
download_dir = Path(r'O:\My Drive')
target_dir = Path('Q:\p_aicore_pf\initze\data\ArcticDEM')

src_elevation = download_dir / 'AI-CORE elevation'
src_slope = download_dir / 'AI-CORE slope'
tgt_elevation = target_dir / 'tiles_rel_el'
tgt_slope = target_dir / 'tiles_slope'

In [ ]:
el_list = list(src_elevation.glob('*.tif'))
for f in tqdm.tqdm_notebook(el_list[:]):
    try:
        shutil.move(str(f), tgt_elevation)
    except:
        print(f'{str(f)} already exists')
        continue

In [ ]:
slope_list = list(src_slope.glob('*.tif'))
for f in tqdm.tqdm_notebook(slope_list[:]):
    try:
        shutil.move(str(f), tgt_slope)
    except:
        print(f'{str(f)} already exists')
        continue